# Задание 2: Автоматическое дифференцирование.

## 2.0. Импорт библиотеки и создание девайса.

In [1]:
import torch

# Введём устройство, на котором будут работать тензоры.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Тензоры на {device}.")

Тензоры на cuda:0.


## 2.1. Простые вычисления с градиентами

In [18]:
x = torch.tensor([1., 2.], requires_grad=True)
y = torch.tensor([3., 4.], requires_grad=True)
z = torch.tensor([5., 6.], requires_grad=True)

def find_gradients(x, y, z):
    """
    Аналитически вычисляет градиенты для тензоров x, y, z
    """
    grad_x = 2 * x + 2 * y * z
    grad_y = 2 * y + 2 * x * z
    grad_z = 2 * z + 2 * y * x

    return grad_x, grad_y, grad_z

f = lambda x, y, z: (x**2 + y**2 + z**2 + 2 * x * y * z).sum()
f(x, y, z).backward()

grad_x_analytic, grad_y_analytic, grad_z_analytic = find_gradients(x, y, z)

print(f"Автоматический x.grad: {x.grad}")
print(f"Автоматический y.grad: {y.grad}")
print(f"Автоматический z.grad: {z.grad}\n")

print(f"Аналитический x.grad: {grad_x_analytic}")
print(f"Аналитический y.grad: {grad_y_analytic}")
print(f"Аналитический z.grad: {grad_z_analytic}")

Автоматический x.grad: tensor([32., 52.])
Автоматический y.grad: tensor([16., 32.])
Автоматический z.grad: tensor([16., 28.])

Аналитический x.grad: tensor([32., 52.], grad_fn=<AddBackward0>)
Аналитический y.grad: tensor([16., 32.], grad_fn=<AddBackward0>)
Аналитический z.grad: tensor([16., 28.], grad_fn=<AddBackward0>)


## 2.2. Градиент функции потерь

In [2]:
def mse(y_true, y_pred):
    return torch.mean((y_pred - y_true) ** 2)

def find_gradients(X, y_true, w, b):
    """
    Аналитически вычисляет градиенты MSE по w и b.
    """
    n = len(y_true)
    y_pred = w * X + b
    
    grad_w = -(2 / n) * X.T @ (y_true - y_pred)
    grad_b = -(2 / n) * (y_true - y_pred)
    
    return grad_w, grad_b

## 2.3. Цепное правило

In [13]:
def func(x): 
    return torch.sin(x**2 + 1)

def find_gradients(x):
    """
    Аналитически вычисляет градиенты с помощью производной функции func.
    """
    return torch.cos(x**2 + 1) * 2 * x

x = torch.tensor([1., 2., 3.], requires_grad=True)
res = func(x)

grad_analytics = find_gradients(x)

grad_auto = torch.autograd.grad(
    outputs=res, 
    inputs=x,
    grad_outputs=torch.ones_like(res),
    create_graph=True
)[0]

print(f"X: {x}\n")
print(f"Аналитический градиент: {grad_analytics}")
print(f"Автоматический градиент: {grad_auto}")

X: tensor([1., 2., 3.], requires_grad=True)

Аналитический градиент: tensor([-0.8323,  1.1346, -5.0344], grad_fn=<MulBackward0>)
Автоматический градиент: tensor([-0.8323,  1.1346, -5.0344], grad_fn=<MulBackward0>)
